##0. Ideation
1. Find-tune CLIP on MNIST
2. Load fine-tuned CLIP and base-CLIP
3. Begin Inference
4. Register Hooks/Intercept Activations to get First layer CLS embedding, last layer CLS embedding, final projected CLIP image embedding
5. Use a classifier head to keep track of the first layer, last layer, and final embedding vectors from the base model and fine-tuned model
    1. base_first -> fine_first (Early low-level shift) - Not as likely. After getting transformation matrix, multiply base_first by it and continue through with rest of layers
    2. base_first -> fine_last (Deep vision changes) - Not as likely. After getting transformation matrix, multiply base_first by it and go through to the projection matrix.
    3. base_first -> fine_final (Total task-level transformation) - Possible. After getting transformation matrix, multiply base_first by it and go straight to classification.
    4. base_last -> fine_last (High-level internal difference) - Possible. After getting transformation matrix, multiply base_last by it and go straight to the projection matrix.
    5. base_last -> fine_final (Final classification alignment) - Possible. After getting transformation matrix, multiply base_last by it and go straight to classification.
    6. **base_final -> fine_final (End-to-end latent mapping) - Most Important. After getting transformation matrix, multiply base_final by it and go straight to classification. **
6. Average out each one, and use least squares regression to calculate it in relation to the fine-tuned model. Use affine transformation to include a bias term to shift to issues. If it doesn’t go well, can also switch to ridge regression to see if it matches it better.
7. Then, use the multiplicative matrix transformation and employ that into the base model itself’s layer right before classification
8. Then, run testing on it and benchmark performance.


##1. Quick Installs for Essential Libraries

In [ ]:
!pip install torch torchvision
!pip install -U transformers datasets
!pip install fifty regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install matplotlib
!pip install -U pillow
%matplotlib inline

In [ ]:
!pip install --force-reinstall --no-cache-dir scipy datasets # Only needed within runpod environment

In [ ]:
!pip install numpy==1.26.4 # only needed for runpod environment

##2. Importing the Libraries

In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
import clip
import numpy as np
from datasets import load_dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

##3. Setting Device, Preparing Data, and Loading CLIP

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device) # https://github.com/openai/CLIP
clip_model.float()
mnist = load_dataset("ylecun/mnist") # https://huggingface.co/datasets/ylecun/mnist
split = mnist["train"].train_test_split(test_size=0.2, seed=66)

train_dataset = split["train"] # 48,000 examples (direct training data from training set)
val_dataset = split["test"] # 12,000 examples (validation set split from training set)
test_dataset = mnist["test"] # 10,000 examples (direct test set)

##4. Pre-Processing MNIST for CLIP

In [ ]:
train_dataset.set_format(type="python", columns=["image", "label"])
val_dataset.set_format(type="python", columns=["image", "label"])
test_dataset.set_format(type="python", columns=["image", "label"])

##5. Dataloaders

In [ ]:
def clip_collate_fn(batch):
  images = []
  labels = []

  for item in batch:
    img = item["image"].convert("RGB")  # Already a PIL Image
    img = preprocess(img)
    images.append(img)
    labels.append(item["label"])

  images = torch.stack(images)
  labels = torch.tensor(labels, dtype=torch.long)

  return {
      "pixel_values": images.to(device),
      "labels": labels.to(device)
  }

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=clip_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=clip_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=clip_collate_fn)

##6. Fine-Tune Prep

In [ ]:
class CLIPClassifier(nn.Module):
  def __init__(self, clip_model, num_classes=10):
    super().__init__()
    self.clip = clip_model
    self.classifier = nn.Linear(self.clip.visual.output_dim, num_classes)

  def forward(self, images):
    image_features = self.clip.encode_image(images)
    logits = self.classifier(image_features)
    return logits

model = CLIPClassifier(clip_model=clip_model).to(device)

In [ ]:
if device == "cpu":
  model.float()

optimizer = optim.Adam(model.parameters(), lr=1e-5)

criterion = nn.CrossEntropyLoss() # Use Contrastive Loss later to see whether it does the same but better. Can test zero shot behavior where we can see if it can identify a photo of 7 without explicitly trainining for it.

EPOCHS = 5
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader) * EPOCHS)

##7. Fine-Tuning CLIP on MNIST

In [ ]:
best_val_loss = float('inf')
best_epoch = -1

for epoch in range(EPOCHS):
  print(f"Epoch {epoch+1}/{EPOCHS} - Best Val Loss: {best_val_loss:.4f} (Epoch {best_epoch})")

  model.train()
  total_train_loss = 0
  train_steps = 0

  for batch in tqdm(train_loader, desc="Training"):
    optimizer.zero_grad()

    images = batch["pixel_values"]
    labels = batch["labels"]

    logits = model(images)
    loss = criterion(logits, labels)

    loss.backward()
    optimizer.step()

    total_train_loss += loss.item()
    train_steps += 1

  avg_train_loss = total_train_loss / train_steps

  # Validation
  model.eval()
  total_val_loss = 0
  val_steps = 0

  with torch.no_grad():
    for batch in tqdm(val_loader, desc="Validation"):
      images = batch["pixel_values"]
      labels = batch["labels"]

      logits = model(images)
      loss = criterion(logits, labels)

      total_val_loss += loss.item()
      val_steps += 1

  avg_val_loss = total_val_loss / val_steps

  print(f"[Epoch {epoch+1}] Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

  if avg_val_loss < best_val_loss:
    best_val_loss = avg_val_loss
    best_epoch = epoch
    torch.save(model.state_dict(), "best_clip_mnist.pt")

  scheduler.step()

torch.save(model.state_dict(), "last_clip_mnist.pt")

##7.5 Testing if Fine-Tuning Actually Worked

In [ ]:
base_CLIP, _ = clip.load("ViT-B/32", device=device)
base_CLIP = base_CLIP.float()
model = CLIPClassifier(clip_model=base_CLIP).to(device)

best_CLIP, _ = clip.load("ViT-B/32", device=device)
best_CLIP = best_CLIP.float()
best_CLIP_MNIST = CLIPClassifier(clip_model=best_CLIP).to(device)
best_CLIP_MNIST.load_state_dict(torch.load("best_clip_mnist.pt", map_location=device)) # map_location tells where to place the model's weights in memory

last_CLIP, _ = clip.load("ViT-B/32", device=device)
last_CLIP = last_CLIP.float()
last_CLIP_MNIST = CLIPClassifier(clip_model=last_CLIP).to(device)
last_CLIP_MNIST.load_state_dict(torch.load("last_clip_mnist.pt", map_location=device)) # map_location tells where to place the model's weights in memory

model.eval()
best_CLIP_MNIST.eval()
last_CLIP_MNIST.eval()

total_test_loss_base = 0
total_base = 0
total_test_loss_best = 0
total_best = 0
total_test_loss_last = 0
total_last = 0

correct_base = 0
correct_best = 0
correct_last = 0
total_samples = 0

with torch.no_grad():
  for batch in tqdm(test_loader, desc="Testing"):
    images = batch["pixel_values"]
    labels = batch["labels"]
    total_samples += labels.size(0)

    # Base model
    logits_base = model(images)
    loss_base = criterion(logits_base, labels)
    total_test_loss_base += loss_base.item()
    total_base += 1

    # Best model
    logits_best = best_CLIP_MNIST(images)
    loss_best = criterion(logits_best, labels)
    total_test_loss_best += loss_best.item()
    total_best += 1

    # Last Model
    logits_last = last_CLIP_MNIST(images)
    loss_last = criterion(logits_last, labels)
    total_test_loss_last += loss_last.item()
    total_last += 1

    # Classification Accuracy
    pred_base = logits_base.argmax(dim=1)
    pred_best = logits_best.argmax(dim=1)
    pred_last = logits_last.argmax(dim=1)

    correct_base += (pred_base == labels).sum().item()
    correct_best += (pred_best == labels).sum().item()
    correct_last += (pred_last == labels).sum().item()

avg_base_loss = total_test_loss_base / total_base
avg_best_loss = total_test_loss_best / total_best
avg_last_loss = total_test_loss_last / total_last

accuracy_base = correct_base / total_samples
accuracy_best = correct_best / total_samples
accuracy_last = correct_last / total_samples
print(f"\nAverage base loss: {avg_base_loss:.4f}, Base Accuracy: {accuracy_base:.4f}")
print(f"Average best loss: {avg_best_loss:.4f}, Best Accuracy: {accuracy_best:.4f}")
print(f"Average last loss: {avg_last_loss:.4f}, Best Accuracy: {accuracy_best:.4f}")